# Modelo para predição de evolução para óbito de pacientes com SRAG

O modelo atual utiliza as bases de dados disponibilizadas pelo [Opendatasus](https://opendatasus.saude.gov.br/).

Os arquivos cvs serão lidos e concatenados em um único dataframe que será salvo no formato parquet.

## Etapa ETL (Extrair, Transformar e Carregar)

In [103]:
import pandas as pd
import os
import datetime
import dateutil
import numpy as np
from epiweeks import Week, Year

pd.set_option('display.max_columns', None)

In [7]:
path = '../'
bases = {}
for item in os.listdir(path):
    full_path = os.path.join(path, item)
    if os.path.isfile(full_path):
        if item.split('.')[1]=='csv':
            bases[item.split('.')[0]] = pd.read_csv(full_path, sep=';',low_memory=False)

In [8]:
colunas = bases['INFLUD21-26-06-2025'].columns.to_list()
for base in bases:
    print(f'As colunas da base {base} é igual ao conjunto padrão(ano 2021)? {colunas == bases[base].columns.to_list()}')


As colunas da base INFLUD21-26-06-2025 é igual ao conjunto padrão(ano 2021)? True
As colunas da base INFLUD22-26-06-2025 é igual ao conjunto padrão(ano 2021)? True
As colunas da base INFLUD23-26-06-2025 é igual ao conjunto padrão(ano 2021)? True
As colunas da base INFLUD24-26-06-2025 é igual ao conjunto padrão(ano 2021)? True
As colunas da base INFLUD25-01-09-2025 é igual ao conjunto padrão(ano 2021)? True


Vamos concatenar as bases gerando conjunto único.

In [9]:
bases_concatenadas = pd.DataFrame()
for base in bases:
    bases_concatenadas = pd.concat([bases_concatenadas, pd.DataFrame(bases[base])], axis=0)

In [10]:
bases_concatenadas.shape

(3093311, 194)

In [11]:
bases_concatenadas.columns.to_list()

['NU_NOTIFIC',
 'DT_NOTIFIC',
 'SEM_NOT',
 'DT_SIN_PRI',
 'SEM_PRI',
 'SG_UF_NOT',
 'ID_REGIONA',
 'CO_REGIONA',
 'ID_MUNICIP',
 'CO_MUN_NOT',
 'CS_SEXO',
 'DT_NASC',
 'NU_IDADE_N',
 'TP_IDADE',
 'COD_IDADE',
 'CS_GESTANT',
 'CS_RACA',
 'CS_ETINIA',
 'CS_ESCOL_N',
 'ID_PAIS',
 'CO_PAIS',
 'SG_UF',
 'ID_RG_RESI',
 'CO_RG_RESI',
 'ID_MN_RESI',
 'CO_MUN_RES',
 'CS_ZONA',
 'NOSOCOMIAL',
 'AVE_SUINO',
 'FEBRE',
 'TOSSE',
 'GARGANTA',
 'DISPNEIA',
 'DESC_RESP',
 'SATURACAO',
 'DIARREIA',
 'VOMITO',
 'OUTRO_SIN',
 'OUTRO_DES',
 'FATOR_RISC',
 'PUERPERA',
 'CARDIOPATI',
 'HEMATOLOGI',
 'SIND_DOWN',
 'HEPATICA',
 'ASMA',
 'DIABETES',
 'NEUROLOGIC',
 'PNEUMOPATI',
 'IMUNODEPRE',
 'RENAL',
 'OBESIDADE',
 'OBES_IMC',
 'OUT_MORBI',
 'MORB_DESC',
 'TABAG',
 'VACINA',
 'DT_UT_DOSE',
 'MAE_VAC',
 'DT_VAC_MAE',
 'M_AMAMENTA',
 'DT_DOSEUNI',
 'DT_1_DOSE',
 'DT_2_DOSE',
 'ANTIVIRAL',
 'TP_ANTIVIR',
 'OUT_ANTIV',
 'DT_ANTIVIR',
 'HOSPITAL',
 'DT_INTERNA',
 'SG_UF_INTE',
 'ID_RG_INTE',
 'CO_RG_INTE',
 'ID_

Algmas variáveeis foram removidas

In [12]:
bases_concatenadas = bases_concatenadas[['NU_NOTIFIC',
 'DT_NOTIFIC',
 'SEM_NOT',
 'DT_SIN_PRI',
 'SEM_PRI',
 'SG_UF_NOT',
 'ID_REGIONA',
 'ID_MUNICIP',
 'CO_MUN_NOT',
 'CS_SEXO',
 'DT_NASC',
 'NU_IDADE_N',
 'TP_IDADE',
 'COD_IDADE',
 'CS_GESTANT',
 'CS_RACA',
 'CS_ESCOL_N',
 'ID_PAIS',
 'SG_UF',
 'ID_RG_RESI',
 'CO_MUN_RES',
 'CS_ZONA',
 'NOSOCOMIAL',
 'AVE_SUINO',
 'FEBRE',
 'TOSSE',
 'GARGANTA',
 'DISPNEIA',
 'DESC_RESP',
 'SATURACAO',
 'DIARREIA',
 'VOMITO',
 'OUTRO_SIN',
 'FATOR_RISC',
 'PUERPERA',
 'CARDIOPATI',
 'HEMATOLOGI',
 'SIND_DOWN',
 'HEPATICA',
 'ASMA',
 'DIABETES',
 'NEUROLOGIC',
 'PNEUMOPATI',
 'IMUNODEPRE',
 'RENAL',
 'OBESIDADE',
 'OBES_IMC',
 'OUT_MORBI',
 'TABAG',
 'VACINA',
 'DT_UT_DOSE',
 'MAE_VAC',
 'DT_VAC_MAE',
 'M_AMAMENTA',
 'DT_DOSEUNI',
 'DT_1_DOSE',
 'DT_2_DOSE',
 'ANTIVIRAL',
 'TP_ANTIVIR',
 'DT_ANTIVIR',
 'HOSPITAL',
 'DT_INTERNA',
 'SG_UF_INTE',
 'ID_RG_INTE',
 'CO_MU_INTE',
 'UTI',
 'DT_ENTUTI',
 'DT_SAIDUTI',
 'SUPORT_VEN',
 'RAIOX_RES',
 'DT_RAIOX',
 'AMOSTRA',
 'DT_COLETA',
 'TP_AMOSTRA',
 'PCR_RESUL',
 'DT_PCR',
 'POS_PCRFLU',
 'TP_FLU_PCR',
 'PCR_FLUASU',
 'PCR_FLUBLI',
 'POS_PCROUT',
 'PCR_VSR',
 'PCR_PARA1',
 'PCR_PARA2',
 'PCR_PARA3',
 'PCR_PARA4',
 'PCR_ADENO',
 'PCR_METAP',
 'PCR_BOCA',
 'PCR_RINO',
 'PCR_OUTRO',
 'CLASSI_FIN',
 'CRITERIO',
 'EVOLUCAO',
 'DT_EVOLUCA',
 'DT_ENCERRA',
 'DT_DIGITA',
 'PCR_SARS2',
 'DOR_ABD',
 'FADIGA',
 'PERD_OLFT',
 'PERD_PALA',
 'TOMO_RES',
 'DT_TOMO',
 'TP_TES_AN',
 'DT_RES_AN',
 'RES_AN',
 'POS_AN_FLU',
 'TP_FLU_AN',
 'POS_AN_OUT',
 'AN_SARS2',
 'AN_VSR',
 'AN_PARA1',
 'AN_PARA2',
 'AN_PARA3',
 'AN_ADENO',
 'AN_OUTRO',
 'TP_AM_SOR',
 'RES_IGG',
 'RES_IGM',
 'RES_IGA',
 'POV_CT',
 'TEM_CPF',
 'ESTRANG',
 'VACINA_COV',
 'DOSE_1_COV',
 'DOSE_2_COV',
 'DOSE_REF',
 'DOSE_2REF',
 'DOSE_ADIC',
 'DOS_RE_BI',
 'TRAT_COV',
 'TIPO_TRAT',
 'DT_TRT_COV',
 'SURTO_SG',
 'CO_DETEC',
 'VG_OMS',
 'VG_MET',
 'VG_DTRES',
 'VG_ENC']]

Vamos converter as datas em formato date

In [13]:
for data in ['DT_NOTIFIC',
             'DT_SIN_PRI',
             'DT_NASC',
             'DT_UT_DOSE',
             'DT_VAC_MAE',
             'DT_DOSEUNI',
             'DT_1_DOSE',
             'DT_2_DOSE',
             'DT_ANTIVIR',
             'DT_INTERNA',
             'DT_ENTUTI',
             'DT_SAIDUTI',
             'DT_RAIOX',
             'DT_COLETA',
             'DT_PCR',
             'DT_EVOLUCA',
             'DT_ENCERRA',
             'DT_DIGITA',
             'DT_TOMO',
             'DT_RES_AN',
             'DT_TRT_COV']:
    bases_concatenadas[data] = pd.to_datetime(bases_concatenadas[data], format='ISO8601')

In [14]:
for data in ['DOSE_1_COV',
             'DOSE_2_COV',
             'DOSE_REF',
             'DOSE_2REF',
             'DOSE_ADIC',
             'DOS_RE_BI',
             'VG_DTRES']:
    bases_concatenadas[data] = pd.to_datetime(bases_concatenadas[data], format='%d/%m/%Y', errors='coerce')
        

In [15]:
for coluna in bases_concatenadas.columns.to_list():
    print(f'{coluna}: {bases_concatenadas[coluna].unique()}') 

NU_NOTIFIC: [  315854073650   315855775774   315862731774 ... 31756700781513
 31756701213039 31756701478780]
DT_NOTIFIC: <DatetimeArray>
['2021-01-28 00:00:00', '2021-07-16 00:00:00', '2021-06-28 00:00:00',
 '2021-02-17 00:00:00', '2021-03-11 00:00:00', '2021-01-26 00:00:00',
 '2021-02-16 00:00:00', '2021-03-10 00:00:00', '2021-05-03 00:00:00',
 '2021-06-15 00:00:00',
 ...
 '2025-08-23 00:00:00', '2025-08-24 00:00:00', '2025-08-25 00:00:00',
 '2025-08-26 00:00:00', '2025-08-27 00:00:00', '2025-08-28 00:00:00',
 '2025-08-29 00:00:00', '2025-08-31 00:00:00', '2025-08-30 00:00:00',
 '2025-09-01 00:00:00']
Length: 1703, dtype: datetime64[ns]
SEM_NOT: [ 4 28 26  7 10 18 24 30 20 11 13 29 38  3 14 35 31 15  8 12  1 19 22  9
  2 48 34 17 33 27 16 21 37  5 42 41 46  6 32 25 51 36 23 49 47 52 39 40
 44 43 50 45]
DT_SIN_PRI: <DatetimeArray>
['2021-01-26 00:00:00', '2021-07-14 00:00:00', '2021-06-05 00:00:00',
 '2021-01-25 00:00:00', '2021-02-15 00:00:00', '2021-03-01 00:00:00',
 '2021-01-20 00:0

In [88]:
hoje = datetime.datetime.now()
bases_concatenadas.to_parquet(f'../bases_concatenadas_{hoje.strftime("%Y-%m-%d_%H-%M-%S")}.parquet')

## Início script após salvamento parquet
Após ter gerado arquivo parquet o script pode ser iniciado neste ponto.

In [69]:
bases_concatenadas = pd.read_parquet('../bases_concatenadas_2025-09-07_20-22-11.parquet')

## Cria variável SRAG 
Utilizamos a definição de caso que é dada por 4 critérios que devem estar presentes:

1. Hospitalização ou óbito
2. Presença de tosse ou dor de garganta
3. Dispnéia ou saturação ($O_2<95\%$) ou desconforto respiratório
4. Febre

In [70]:
condicao = ((bases_concatenadas['HOSPITAL'] == 1) | (bases_concatenadas['EVOLUCAO'] == 2)) & ((bases_concatenadas['TOSSE'] == 1) | (bases_concatenadas['GARGANTA'] == 1)) & ((bases_concatenadas['DISPNEIA'] == 1) | (bases_concatenadas['SATURACAO'] == 1) | (bases_concatenadas['DESC_RESP'] == 1)) & ((bases_concatenadas['FEBRE'] == 1)) 
bases_concatenadas['caso_srag'] = np.where(condicao, 1, 0)

In [71]:
srag = bases_concatenadas.loc[bases_concatenadas['caso_srag']==1].copy()

## Definindo casos de influenza

Vamos estabelecer casos de Influenza A:

In [78]:
# H1N1
condicao = (srag['PCR_FLUASU'] == 1)
srag['influenza_h1n1'] = np.where(condicao,1,0)

In [79]:
# H3N2
condicao = (srag['PCR_FLUASU'] == 2)
srag['influenza_h3n2'] = np.where(condicao,1,0)

In [80]:
# Influenza não subtipável
condicao = (srag['PCR_FLUASU'] == 4)
srag['influenza_n_subtipavel'] = np.where(condicao,1,0)

In [81]:
# Influenza não subtipada
condicao = (srag['PCR_FLUASU'] == 3) | (
    ((srag['TP_FLU_AN']==1) | (srag['TP_FLU_PCR']==1)) & 
    (srag['PCR_FLUASU'] != 1) & (srag['PCR_FLUASU'] != 2) & (srag['PCR_FLUASU'] != 4))
srag['influenza_A_n_subtipada'] = np.where(condicao, 1, 0)

In [86]:
# Total influenza A
condicao = ((srag['influenza_h1n1'] == 1) | 
            (srag['influenza_h3n2'] == 1) |
            (srag['influenza_n_subtipavel'] == 1) | 
            (srag['influenza_A_n_subtipada'] == 1))
srag['influenza_A'] = np.where(condicao, 1, 0)

Influenza B

In [87]:
# VICTORIA
condicao = (srag['PCR_FLUBLI'] == 1)
srag['influenza_b_victoria'] = np.where(condicao,1,0)

In [88]:
# YAMAGATHA
condicao = (srag['PCR_FLUBLI'] == 2)
srag['influenza_b_yamagatha'] = np.where(condicao,1,0)

In [91]:
# Influenza B inconclusivo
condicao = (((srag['TP_FLU_AN']==2) | (srag['TP_FLU_PCR']==2)) & 
    (srag['PCR_FLUBLI'] != 1) & (srag['PCR_FLUBLI'] != 2))
srag['influenza_b_inconclusivo'] = np.where(condicao,1,0)

In [92]:
# Influenza B
condicao = ((srag['influenza_b_victoria']==1) | 
            (srag['influenza_b_yamagatha']==1) |
            (srag['influenza_b_inconclusivo']==1))
srag['influenza_B'] = np.where(condicao,1,0)

Casos de influenza (A+B)

In [97]:
condicao = ((srag['influenza_A']==1) |
            (srag['influenza_B']==1))
srag['influenza'] = np.where(condicao,1,0)

Vamos filtrar casos de SRAG por influenza

In [100]:
influenza = srag.loc[srag['influenza']==1]

In [106]:
influenza.head()

,NU_NOTIFIC,DT_NOTIFIC,SEM_NOT,DT_SIN_PRI,SEM_PRI,SG_UF_NOT,ID_REGIONA,ID_MUNICIP,CO_MUN_NOT,CS_SEXO,DT_NASC,NU_IDADE_N,TP_IDADE,COD_IDADE,CS_GESTANT,CS_RACA,CS_ESCOL_N,ID_PAIS,SG_UF,ID_RG_RESI,CO_MUN_RES,CS_ZONA,NOSOCOMIAL,AVE_SUINO,FEBRE,TOSSE,GARGANTA,DISPNEIA,DESC_RESP,SATURACAO,DIARREIA,VOMITO,OUTRO_SIN,FATOR_RISC,PUERPERA,CARDIOPATI,HEMATOLOGI,SIND_DOWN,HEPATICA,ASMA,DIABETES,NEUROLOGIC,PNEUMOPATI,IMUNODEPRE,RENAL,OBESIDADE,OBES_IMC,OUT_MORBI,TABAG,VACINA,DT_UT_DOSE,MAE_VAC,DT_VAC_MAE,M_AMAMENTA,DT_DOSEUNI,DT_1_DOSE,DT_2_DOSE,ANTIVIRAL,TP_ANTIVIR,DT_ANTIVIR,HOSPITAL,DT_INTERNA,SG_UF_INTE,ID_RG_INTE,CO_MU_INTE,UTI,DT_ENTUTI,DT_SAIDUTI,SUPORT_VEN,RAIOX_RES,DT_RAIOX,AMOSTRA,DT_COLETA,TP_AMOSTRA,PCR_RESUL,DT_PCR,POS_PCRFLU,TP_FLU_PCR,PCR_FLUASU,PCR_FLUBLI,POS_PCROUT,PCR_VSR,PCR_PARA1,PCR_PARA2,PCR_PARA3,PCR_PARA4,PCR_ADENO,PCR_METAP,PCR_BOCA,PCR_RINO,PCR_OUTRO,CLASSI_FIN,CRITERIO,EVOLUCAO,DT_EVOLUCA,DT_ENCERRA,DT_DIGITA,PCR_SARS2,DOR_ABD,FADIGA,PERD_OLFT,PERD_PALA,TOMO_RES,DT_TOMO,TP_TES_AN,DT_RES_AN,RES_AN,POS_AN_FLU,TP_FLU_AN,POS_AN_OUT,AN_SARS2,AN_VSR,AN_PARA1,AN_PARA2,AN_PARA3,AN_ADENO,AN_OUTRO,TP_AM_SOR,RES_IGG,RES_IGM,RES_IGA,POV_CT,TEM_CPF,ESTRANG,VACINA_COV,DOSE_1_COV,DOSE_2_COV,DOSE_REF,DOSE_2REF,DOSE_ADIC,DOS_RE_BI,TRAT_COV,TIPO_TRAT,DT_TRT_COV,SURTO_SG,CO_DETEC,VG_OMS,VG_MET,VG_DTRES,VG_ENC,caso_srag,influenza_h1n1,influenza_h3n2,influenza_n_subtipavel,influenza_A_n_subtipada,influenza_A,influenza_b_victoria,influenza_b_yamagatha,influenza_b_inconclusivo,influenza_B,influenza
5472,316103848614,2021-01-11,2,2021-01-04,1,SP,GVE I CAPITAL,SAO PAULO,355030.0,F,2020-06-12,7,2,2007,6,4,5.0,BRASIL,SP,GVE I CAPITAL,355030.0,1.0,NaN,NaN,1.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaT,NaT,NaT,1.0,1.0,2021-01-09,1.0,2021-01-07,SP,GVE I CAPITAL,355030.0,1.0,2021-01-07,NaT,2.0,6.0,NaT,1.0,2021-01-07,1.0,2.0,2021-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,2021-01-12,2021-01-13,2021-01-11,NaN,2.0,2.0,2.0,2.0,6.0,NaT,1.0,2021-01-08,1.0,1.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaT,NaN,1,0,0,0,0,0,0,0,1,1,1
7941,316104707580,2021-01-12,2,2021-01-08,1,GO,OESTE I,IPORA,521020.0,F,1945-12-01,75,3,3075,5,1,1.0,BRASIL,GO,OESTE I,521020.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaT,NaT,NaT,2.0,NaN,NaT,1.0,2021-01-11,None,None,NaN,2.0,NaT,NaT,2.0,6.0,NaT,1.0,2021-01-11,1.0,4.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,1.0,1.0,2021-02-17,2021-01-12,2021-01-12,NaN,2.0,2.0,2.0,2.0,6.0,NaT,1.0,2021-01-11,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaT,NaN,1,0,0,0,1,1,0,0,0,0,1
29382,316111517955,2021-01-17,3,2021-01-03,1,SP,GVE XVII CAMPINAS,CAMPINAS,350950.0,M,1954-03-18,66,3,3066,6,1,4.0,BRASIL,SP,GVE XVII CAMPINAS,350950.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,NaN,1.0,NaN,1.0,NaT,NaN,NaT,NaN,NaT,NaT,NaT,1.0,1.0,2021-01-17,1.0,2021-01-17,SP,GVE XVII CAMPINAS,350950.0,1.0,2021-01-17,2021-01-30,2.0,6.0,NaT,1.0,2021-01-11,1.0,1.0,2021-01-11,1.0,1.0,3.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,1.0,1.0,2021-01-31,2021-02-18,2021-01-20,1.0,1.0,2.0,2.0,2.0,1.0,2021-01-17,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,2.0,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaT,NaN,1,0,0,0,1,1,0,0,0,0,1
35455,316113074827,2021-12-23,51,2021-12-17,50,SP,GVE I CAPITAL,SAO PAULO,355030.0,M,1946-08-28,74,3,3074,6,1,NaN,BRASIL,SP,GVE I CAPITAL,355030.0,1.0,2.0,2.0,1.0,1.0,NaN,1.0,1.0,1.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaT,NaN,NaT,NaN,NaT,NaT,NaT,1.0,1.0,2021-12-22,1.0,2021-12-19,SP,GVE I CAPITAL,355030.0,2.0,NaT,N

## Selecionando variáveis de interesse

In [107]:
influenza = influenza[['DT_NOTIFIC',
                     'DT_SIN_PRI',
                     'SG_UF_NOT',
                     'CO_MUN_NOT',
                     'CS_SEXO',
                     'DT_NASC',
                     'NU_IDADE_N',
                     'TP_IDADE',
                     'CS_GESTANT',
                     'CS_RACA',
                     'CS_ESCOL_N',
                     'ID_PAIS',
                     'SG_UF',
                     'CO_MUN_RES',
                     'CS_ZONA',
                     'NOSOCOMIAL',
                     'AVE_SUINO',
                     'FEBRE',
                     'TOSSE',
                     'GARGANTA',
                     'DISPNEIA',
                     'DESC_RESP',
                     'SATURACAO',
                     'DIARREIA',
                     'VOMITO',
                     'OUTRO_SIN',
                     'PUERPERA',
                     'CARDIOPATI',
                     'HEMATOLOGI',
                     'SIND_DOWN',
                     'HEPATICA',
                     'ASMA',
                     'DIABETES',
                     'NEUROLOGIC',
                     'PNEUMOPATI',
                     'IMUNODEPRE',
                     'RENAL',
                     'OBESIDADE',
                     'OBES_IMC',
                     'TABAG',
                     'VACINA',
                     'DT_UT_DOSE',
                     'MAE_VAC',
                     'DT_VAC_MAE',
                     'M_AMAMENTA',
                     'DT_DOSEUNI',
                     'DT_1_DOSE',
                     'DT_2_DOSE',
                     'ANTIVIRAL',
                     'TP_ANTIVIR',
                     'DT_ANTIVIR',
                     'HOSPITAL',
                     'DT_INTERNA',
                     'SG_UF_INTE',
                     'CO_MU_INTE',
                     'UTI',
                     'DT_ENTUTI',
                     'DT_SAIDUTI',
                     'SUPORT_VEN',
                     'RAIOX_RES',
                     'DT_RAIOX',
                     'AMOSTRA',
                     'DT_COLETA',
                     'TP_AMOSTRA',
                     'PCR_RESUL',
                     'DT_PCR',
                     'POS_PCRFLU',
                     'TP_FLU_PCR',
                     'PCR_FLUASU',
                     'PCR_FLUBLI',
                     'POS_PCROUT',
                     'PCR_VSR',
                     'PCR_PARA1',
                     'PCR_PARA2',
                     'PCR_PARA3',
                     'PCR_PARA4',
                     'PCR_ADENO',
                     'PCR_METAP',
                     'PCR_BOCA',
                     'PCR_RINO',
                     'PCR_OUTRO',
                     'EVOLUCAO',
                     'DT_EVOLUCA',
                     'DT_ENCERRA',
                     'PCR_SARS2',
                     'DOR_ABD',
                     'FADIGA',
                     'PERD_OLFT',
                     'PERD_PALA',
                     'TOMO_RES',
                     'DT_TOMO',
                     'TP_TES_AN',
                     'DT_RES_AN',
                     'RES_AN',
                     'POS_AN_FLU',
                     'TP_FLU_AN',
                     'POS_AN_OUT',
                     'AN_SARS2',
                     'AN_VSR',
                     'AN_PARA1',
                     'AN_PARA2',
                     'AN_PARA3',
                     'AN_ADENO',
                     'AN_OUTRO',
                     'POV_CT',
                     'SURTO_SG',
                     ]]

In [108]:
hoje = datetime.datetime.now()
influenza.to_parquet(f'../influenza_{hoje.strftime("%Y-%m-%d_%H-%M-%S")}.parquet')